In [10]:
# !pip install pandas
# !pip install scikit-learn

In [24]:
import warnings
warnings.filterwarnings("ignore")

In [25]:
import pandas as pd

df = pd.read_csv("../data/MachineLearningRating_v3.txt", sep="|")


print("Shape:", df.shape)
df.head()

Shape: (1000098, 52)


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [26]:
df.info()
df.describe(include='all')
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000098 entries, 0 to 1000097
Data columns (total 52 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   UnderwrittenCoverID       1000098 non-null  int64  
 1   PolicyID                  1000098 non-null  int64  
 2   TransactionMonth          1000098 non-null  object 
 3   IsVATRegistered           1000098 non-null  bool   
 4   Citizenship               1000098 non-null  object 
 5   LegalType                 1000098 non-null  object 
 6   Title                     1000098 non-null  object 
 7   Language                  1000098 non-null  object 
 8   Bank                      854137 non-null   object 
 9   AccountType               959866 non-null   object 
 10  MaritalStatus             991839 non-null   object 
 11  Gender                    990562 non-null   object 
 12  Country                   1000098 non-null  object 
 13  Province                  1

UnderwrittenCoverID               0
PolicyID                          0
TransactionMonth                  0
IsVATRegistered                   0
Citizenship                       0
LegalType                         0
Title                             0
Language                          0
Bank                         145961
AccountType                   40232
MaritalStatus                  8259
Gender                         9536
Country                           0
Province                          0
PostalCode                        0
MainCrestaZone                    0
SubCrestaZone                     0
ItemType                          0
mmcode                          552
VehicleType                     552
RegistrationYear                  0
make                            552
Model                           552
Cylinders                       552
cubiccapacity                   552
kilowatts                       552
bodytype                        552
NumberOfDoors               

In [27]:
# Convert TransactionMonth to datetime
df["TransactionMonth"] = pd.to_datetime(df["TransactionMonth"])

In [28]:
df = df.drop(columns=["CrossBorder", "NumberOfVehiclesInFleet"])

### Handling Missing Values

In [29]:
# Missing values before 
df['CustomValueEstimate'].isnull().sum()  

np.int64(779642)

In [30]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split

# Fill Simple Columns
def fill_simple_values(df):
    df['Gender'].fillna(df['Gender'].mode()[0], inplace=True)
    df['MaritalStatus'].fillna(df['MaritalStatus'].mode()[0], inplace=True)
    df['Bank'].fillna("Unknown", inplace=True)
    df['AccountType'].fillna("Unknown", inplace=True)
    return df


# Predict Binary Columns
def predict_binary_column(df, target_col, features):
    print(f"Imputing {target_col}...")

    # Convert Yes/No to binary
    df[target_col] = df[target_col].map({'Yes': 1, 'No': 0})

    # Create subset where target and features are not missing
    df_subset = df[features + [target_col]].dropna()

    # Split into known and missing
    train_data = df_subset[df_subset[target_col].notnull()]
    predict_data = df[df[target_col].isnull()]

    # One-hot encode features
    X = pd.get_dummies(train_data[features])
    y = train_data[target_col]

    X_pred = pd.get_dummies(predict_data[features])
    X_pred = X_pred.reindex(columns=X.columns, fill_value=0)

    # Train classifier
    model = RandomForestClassifier(n_estimators=50, max_depth=8, random_state=42)
    model.fit(X, y)

    # Predict
    predicted = model.predict(X_pred)

    # Fill predictions back
    df.loc[df[target_col].isnull(), target_col] = predicted

    return df


# Predict CustomValueEstimate

def predict_custom_value(df):
    print("Imputing CustomValueEstimate")

    features = [
        'make', 'Province', 'bodytype',
        'RegistrationYear', 'SumInsured', 'kilowatts', 'cubiccapacity'
    ]

    # Clean training data
    df_train = df[df['CustomValueEstimate'].notnull()].dropna(subset=features)
    df_missing = df[df['CustomValueEstimate'].isnull()].dropna(subset=features)

    # Save the indexes of rows we're predicting
    missing_indexes = df_missing.index

    # Sample only 10k rows to avoid memory issues
    df_train_sample = df_train.sample(10000, random_state=42)

    # Combine for consistent encoding
    df_all = pd.concat([df_train_sample[features], df_missing[features]])
    df_encoded = pd.get_dummies(df_all)

    # Split into train and predict sets
    X_train = df_encoded.iloc[:len(df_train_sample), :]
    X_pred = df_encoded.iloc[len(df_train_sample):]
    y_train = df_train_sample['CustomValueEstimate']

    # Train model
    from sklearn.ensemble import RandomForestRegressor
    model = RandomForestRegressor(n_estimators=50, max_depth=8, random_state=42)
    model.fit(X_train, y_train)

    # Predict
    predicted = model.predict(X_pred)

    # Assign back using matching indexes
    df.loc[missing_indexes, 'CustomValueEstimate'] = predicted

    print("CustomValueEstimate imputation complete.")
    return df



def run_full_imputation(df):
    # Fill simple columns
    df = fill_simple_values(df)

    # Common features to use in all imputations
    features = [
    'make', 'Province', 'bodytype',
    'RegistrationYear', 'SumInsured', 'kilowatts', 'cubiccapacity'
]

    # Predict binary targets
    for col in ['WrittenOff', 'Rebuilt', 'Converted']:
        df = predict_binary_column(df, col, features)

    # Predict CustomValueEstimate
    df = predict_custom_value(df)

    return df


In [31]:
df = run_full_imputation(df)

Imputing WrittenOff...
Imputing Rebuilt...
Imputing Converted...
Imputing CustomValueEstimate
CustomValueEstimate imputation complete.


In [32]:
# After handling missing values 
df['CustomValueEstimate'].isnull().sum() 

np.int64(552)

In [33]:
# Final clean-up after model-based predictions
df['CustomValueEstimate'].fillna(df['CustomValueEstimate'].median(), inplace=True)

In [34]:
df['CustomValueEstimate'].isnull().sum() 

np.int64(0)

In [35]:
df.isnull().sum()

UnderwrittenCoverID              0
PolicyID                         0
TransactionMonth                 0
IsVATRegistered                  0
Citizenship                      0
LegalType                        0
Title                            0
Language                         0
Bank                             0
AccountType                      0
MaritalStatus                    0
Gender                           0
Country                          0
Province                         0
PostalCode                       0
MainCrestaZone                   0
SubCrestaZone                    0
ItemType                         0
mmcode                         552
VehicleType                    552
RegistrationYear                 0
make                           552
Model                          552
Cylinders                      552
cubiccapacity                  552
kilowatts                      552
bodytype                       552
NumberOfDoors                  552
VehicleIntroDate    

In [36]:
# categorical features
categorical_car_cols = ['make', 'Model', 'bodytype', 'VehicleType', 'mmcode']
for col in categorical_car_cols:
    df[col] = df[col].fillna('Unknown')

# numerical features
numerical_car_cols = ['kilowatts', 'cubiccapacity', 'Cylinders', 'NumberOfDoors']
for col in numerical_car_cols:
    # Ensure values are numeric and clean any formatting issues
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)
    df[col] = df[col].fillna(df[col].median())


if df['VehicleIntroDate'].isnull().sum() > 0:
    df['VehicleIntroDate'] = df['VehicleIntroDate'].fillna(df['VehicleIntroDate'].mode()[0])


df['NewVehicle'] = df['NewVehicle'].fillna('Unknown')

df['CapitalOutstanding'] = (
    df['CapitalOutstanding']
    .astype(str)
    .str.replace(",", ".")  # replace comma decimal separator
    .str.replace(r"[^\d.]", "", regex=True)  # remove non-numeric characters
)
df['CapitalOutstanding'] = pd.to_numeric(df['CapitalOutstanding'], errors='coerce')
df['CapitalOutstanding'] = df['CapitalOutstanding'].fillna(df['CapitalOutstanding'].median())


In [37]:
df.isnull().sum()

UnderwrittenCoverID         0
PolicyID                    0
TransactionMonth            0
IsVATRegistered             0
Citizenship                 0
LegalType                   0
Title                       0
Language                    0
Bank                        0
AccountType                 0
MaritalStatus               0
Gender                      0
Country                     0
Province                    0
PostalCode                  0
MainCrestaZone              0
SubCrestaZone               0
ItemType                    0
mmcode                      0
VehicleType                 0
RegistrationYear            0
make                        0
Model                       0
Cylinders                   0
cubiccapacity               0
kilowatts                   0
bodytype                    0
NumberOfDoors               0
VehicleIntroDate            0
CustomValueEstimate         0
AlarmImmobiliser            0
TrackingDevice              0
CapitalOutstanding          0
NewVehicle